# Part 2 - VacationPy


In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("C:/Users/sooki/OneDrive/Desktop/api_challenge/output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,-1.91,93,100,5.66,CA,1699692075
1,1,albany,42.6001,-73.9662,5.23,79,33,2.66,US,1699691857
2,2,happy valley-goose bay,53.3168,-60.3315,-2.83,74,75,3.60,CA,1699692219
3,3,mazara del vallo,37.6641,12.5880,20.97,59,50,5.94,IT,1699692220
4,4,port-aux-francais,-49.3500,70.2167,2.70,66,69,10.01,TF,1699691952


## Step 1
- Create a map that displays a point for every city in the city_data_df DataFrame as shown in the following image. 
- The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points("Lng",
                                               "Lat", 
                                               geo = True,
                                               color = "City",
                                               alpha = 0.5,
                                               size = "Humidity",
                                               tiles = "EsriNatGeo")

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2
- Narrow down the city_data_df DataFrame to find your ideal weather condition. 


- **SELECTED weather condition:**
    - A max temperature lower than 30 degrees but higher than 17 degrees
    - Wind speed less than 3.5 m/s
    - Zero cloudiness

In [48]:
# Narrow down cities that fit criteria 
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 30)
    & (city_data_df["Max Temp"] > 17)
    & (city_data_df["Cloudiness"] == 0)
    & (city_data_df["Wind Speed"] < 3.5)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,qaisumah,28.3112,46.1273,27.98,32,0,3.09,SA,1699692238
37,37,hawaiian paradise park,19.5933,-154.9731,22.24,93,0,1.54,US,1699691981
90,90,goondiwindi,-28.5500,150.3167,28.54,28,0,1.53,AU,1699692286
238,238,tamanrasset,22.7850,5.5228,24.95,10,0,1.03,DZ,1699692401
264,264,amarante do maranhao,-5.5667,-46.7422,23.00,79,0,2.14,BR,1699692423
286,286,walvis bay,-22.9575,14.5053,21.53,60,0,2.06,NaN,1699692439
290,290,boa vista,2.8197,-60.6733,26.99,74,0,3.09,BR,1699692155
291,291,marco island,25.9363,-81.7157,23.32,96,0,1.54,US,1699692442
325,325,ghat,24.9647,10.1728,26.03,12,0,1.95,LY,1699692468
346,346,hilo,19.7297,-155.0900,23.68,85,0,1.54,US,1699692367


## Step 3
- Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,qaisumah,SA,28.3112,46.1273,32,
37,hawaiian paradise park,US,19.5933,-154.9731,93,
90,goondiwindi,AU,-28.5500,150.3167,28,
238,tamanrasset,DZ,22.7850,5.5228,10,
264,amarante do maranhao,BR,-5.5667,-46.7422,79,
286,walvis bay,NaN,-22.9575,14.5053,60,
290,boa vista,BR,2.8197,-60.6733,74,
291,marco island,US,25.9363,-81.7157,96,
325,ghat,LY,24.9647,10.1728,12,
346,hilo,US,19.7297,-155.0900,85,


## Step 4
- For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search...")
print("------------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search...
------------------------
qaisumah - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
goondiwindi - nearest hotel: Queensland Hotel
tamanrasset - nearest hotel: فندق الأمان
amarante do maranhao - nearest hotel: No hotel found
walvis bay - nearest hotel: Atlantic
boa vista - nearest hotel: Aipana Plaza Hotel
marco island - nearest hotel: JW Marriott Marco Island Beach Resort
ghat - nearest hotel: فندق تاسيلي
hilo - nearest hotel: Dolphin Bay Hotel
metlili chaamba - nearest hotel: No hotel found
senador pompeu - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
timimoun - nearest hotel: أناف
armacao de buzios - nearest hotel: Vila da Santa
chipinge - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
vila velha - nearest hotel: Hotel Prainha


,City,Country,Lat,Lng,Humidity,Hotel Name
26,qaisumah,SA,28.3112,46.1273,32,No hotel found
37,hawaiian paradise park,US,19.5933,-154.9731,93,No hotel found
90,goondiwindi,AU,-28.5500,150.3167,28,Queensland Hotel
238,tamanrasset,DZ,22.7850,5.5228,10,فندق الأمان
264,amarante do maranhao,BR,-5.5667,-46.7422,79,No hotel found
286,walvis bay,NaN,-22.9575,14.5053,60,Atlantic
290,boa vista,BR,2.8197,-60.6733,74,Aipana Plaza Hotel
291,marco island,US,25.9363,-81.7157,96,JW Marriott Marco Island Beach Resort
325,ghat,LY,24.9647,10.1728,12,فندق تاسيلي
346,hilo,US,19.7297,-155.0900,85,Dolphin Bay Hotel


## Step 5
- Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.9,
                                   size = "Humidity",
                                   tiles = "EsriNatGeo",
                                   hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)